<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL-Tutorials/blob/master/1-Introduction/Stock_NeurIPS2018_SB3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a id='0'></a>
# Part 1. Task Discription

We train a DRL agent for stock trading. This task is modeled as a Markov Decision Process (MDP), and the objective function is maximizing (expected) cumulative return.

We specify the state-action-reward as follows:

* **State s**: The state space represents an agent's perception of the market environment. Just like a human trader analyzing various information, here our agent passively observes many features and learns by interacting with the market environment (usually by replaying historical data).

* **Action a**: The action space includes allowed actions that an agent can take at each state. For example, a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying. When an action operates multiple shares, a ∈{−k, ..., −1, 0, 1, ..., k}, e.g.. "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* **Reward function r(s, a, s′)**: Reward is an incentive for an agent to learn a better policy. For example, it can be the change of the portfolio value when taking a at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio values at state s′ and s, respectively


**Market environment**: 30 consituent stocks of Dow Jones Industrial Average (DJIA) index. Accessed at the starting date of the testing period.


The data for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Install Python Packages

<a id='1.1'></a>
## 2.1. Install packages


In [1]:
## install required packages

if True:
    # installing packages
    !pip install swig
    !pip install wrds
    !pip install pyportfolioopt
    !pip install trading_calendars
    !pip install alpaca_trade_api
    !pip install ccxt
    !pip install jqdatasdk

    !pip install lz4
    !pip install tensorboardX
    !pip install gputil
    #!pip install pyfolio-reloaded  #original pyfolio no longer maintained
    !pip install optuna
    !pip install plotly
    !pip install ipywidgets
    !pip install -U kaleido 


## install finrl library
!pip install -q condacolab
import condacolab
condacolab.install()
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 87.9 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.10
    Uninstalling SQLAlchemy-2.0.10:
      Successfully uninstalled SQLAlchemy-2.0.10
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 3.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.7/108.7 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ...


<a id='1.2'></a>
## 2.2. A list of Python packages 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [10]:
#Importing the libraries
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime
import optuna
from pathlib import Path
#from google.colab import files
%matplotlib inline
from finrl import config
from finrl import config_tickers
#from optuna.integration import PyTorchLightningPruningCallback
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.meta.env_stock_trading.env_stocktrading_np import StockTradingEnv as StockTradingEnv_numpy
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.agents.rllib.models import DRLAgent as DRLAgent_rllib
from finrl.meta.data_processor import DataProcessor
import joblib
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
import ray
from pprint import pprint
#import kaleido



import itertools

import torch
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(f'Torch device: {device}')

/usr/local/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


Torch device: cuda


In [11]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

## Collecting data and preprocessing

In [12]:
#config_tickers.DOW_30_TICKER = ["PYPL"]

In [13]:
#Custom ticker list dataframe download
#TODO save df to avoid download
path_pf = '/content/ticker_data.csv'
if Path(path_pf).is_file():
  print('Reading ticker data')
  df = pd.read_csv(path_pf)
  
else:
  print('Downloading ticker data')
  ticker_list = config_tickers.DOW_30_TICKER
  df = YahooDownloader(start_date = '2009-01-01',
                     end_date = '2023-04-30',
                     ticker_list = ticker_list).fetch_data()
  df.to_csv('ticker_data.csv')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [14]:
def create_processed_full(processed):
  list_ticker = processed["tic"].unique().tolist()
  list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
  combination = list(itertools.product(list_date,list_ticker))

  processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
  processed_full = processed_full[processed_full['date'].isin(processed['date'])]
  processed_full = processed_full.sort_values(['date','tic'])

  processed_full = processed_full.fillna(0)
  processed_full.sort_values(['date','tic'],ignore_index=True).head(5)

  processed_full.to_csv('processed_full.csv')
  return processed_full

In [15]:
#You can add technical indicators and turbulence factor to dataframe
#Just set the use_technical_indicator=True, use_vix=True and use_turbulence=True
def create_techind():
  fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

  processed = fe.preprocess_data(df)
  return processed

In [16]:
#Load price and technical indicator data from file if available
path_pf = '/content/processed_full.csv'
if Path(path_pf).is_file():
  print('Reading processed_full data')
  processed_full = pd.read_csv(path_pf)

else:
  print('Creating processed_full file')
  processed=create_techind()
  processed_full=create_processed_full(processed)

Creating processed_full file
Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (3604, 8)
Successfully added vix
Successfully added turbulence index


In [20]:
date_col = "date"
tic_col = "tic"

init_train_trade_data = processed_full.sort_values([date_col, tic_col])

init_train_trade_data = processed_full.fillna(0)

init_train_data = data_split(
    init_train_trade_data, '2010-01-01', '2022-05-01')
init_trade_data = data_split(
    init_train_trade_data, '2022-05-01','2023-04-30')

print(f'Number of training samples: {len(init_train_data)}')
print(f'Number of testing samples: {len(init_train_trade_data)}')

Number of training samples: 89987
Number of testing samples: 104516


In [21]:
init_train_data.shape

(89987, 18)

## Creating Environment

In [22]:
class Portfolio:
    def __init__(self):
        self.num_shares = 0
        self.total_cost = 0.0
        self.avg_cost = 0.0

    def buy(self, num_shares, price):
        self.total_cost += num_shares * price
        self.num_shares += num_shares
        self.avg_cost = self.total_cost / self.num_shares

    def sell(self, num_shares):
        self.total_cost -= self.avg_cost * num_shares
        self.num_shares -= num_shares

    def get_num_shares(self):
        return self.num_shares

    def get_total_cost(self):
        return self.total_cost

    def get_avg_cost(self):
        return self.avg_cost

# Example usage
portfolio = Portfolio()

# Buy 50 shares at $126.81
portfolio.buy(50, 126.81)

# Sell 25 shares
portfolio.sell(25)

# Buy 30 shares at $125.03
portfolio.buy(30, 125.03)

# Sell 10 shares
portfolio.sell(10)


print("Number of shares: ", portfolio.get_num_shares())
print("Total cost: ", round(portfolio.get_total_cost(), 2))
print("Average cost: ", round(portfolio.get_avg_cost(), 2))


Number of shares:  45
Total cost:  5662.76
Average cost:  125.84


In [166]:
from __future__ import annotations
import copy
from typing import List

import gym
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(linewidth=np.inf)
import pandas as pd
from gym import spaces
from gym.utils import seeding
from stable_baselines3.common.vec_env import DummyVecEnv

matplotlib.use("Agg")

# from stable_baselines3.common.logger import Logger, KVWriter, CSVOutputFormat


class GokuEnv(gym.Env):
    """A stock trading environment for OpenAI gym"""

    metadata = {"render.modes": ["human"]}

    def __init__(
        self,
        df: pd.DataFrame,
        stock_dim: int,
        hmax: int,
        initial_amount: int,
        num_stock_shares: list[int],
        buy_cost_pct: list[float],
        sell_cost_pct: list[float],
        reward_scaling: float,
        state_space: int,
        action_space: int,
        tech_indicator_list: list[str],
        turbulence_threshold=None,
        risk_indicator_col="turbulence",
        make_plots: bool = False,
        print_verbosity=10,
        day=0,
        initial=True,
        previous_state=[],
        model_name="",
        mode="",
        iteration="",
    ):
        self.day = day
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.num_stock_shares = num_stock_shares
        self.initial_amount = initial_amount  # get the initial cash
        self.buy_cost_pct = buy_cost_pct
        self.sell_cost_pct = sell_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list
        self.action_space = spaces.Box(low=-1, high=1, shape=(self.action_space,))
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(self.state_space,)
        )
        self.data = self.df.loc[self.day, :]
        self.terminal = False
        self.make_plots = make_plots
        self.print_verbosity = print_verbosity
        self.turbulence_threshold = turbulence_threshold
        self.risk_indicator_col = risk_indicator_col
        self.initial = initial
        self.previous_state = previous_state
        self.model_name = model_name
        self.mode = mode
        self.iteration = iteration
        # initalize state
        self.state = self._initiate_state()

        # initialize reward
        self.reward = 0
        self.turbulence = 0
        self.cost = 0
        self.trades = 0
        self.episode = 0
        # memorize all the total balance change
        self.asset_memory = [
            self.initial_amount
            + np.sum(
                np.array(self.num_stock_shares)
                * np.array(self.state[1 : 1 + self.stock_dim])
            )
        ]  # the initial total asset is calculated by cash + sum (num_share_stock_i * price_stock_i)
        self.rewards_memory = []
        self.actions_memory = []
        self.state_memory = (
            []
        )  # we need sometimes to preserve the state in the middle of trading process
        self.date_memory = [self._get_date()]
        #         self.logger = Logger('results',[CSVOutputFormat])
        # self.reset()
        self._seed()
        self.total_price = np.array([0.0] * 29)
        self.avg_price = np.array([0.0] * 29)
        self.total_stockss = np.array([0.0] * 29)
        self.frame  = pd.DataFrame()
        self.portfolios = [Portfolio() for _ in range(29)]
        self.count = 0

    def _sell_stock(self, index, action):
        def _do_sell_normal():
            if (
                self.state[index + 2 * self.stock_dim + 1] != True
            ):  # check if the stock is able to sell, for simlicity we just add it in techical index
                # if self.state[index + 1] > 0: # if we use price<0 to denote a stock is unable to trade in that day, the total asset calculation may be wrong for the price is unreasonable
                # Sell only if the price is > 0 (no missing data in this particular date)
                # perform sell action based on the sign of the action
                if self.state[index + self.stock_dim + 1] > 0:
                    # Sell only if current asset is > 0
                    sell_num_shares = min(
                        abs(action), self.state[index + self.stock_dim + 1]
                    )
                    sell_amount = (
                        self.state[index + 1]
                        * sell_num_shares
                        * (1 - self.sell_cost_pct[index])
                    )
                    # update balance
                    self.state[0] += sell_amount

                    self.state[index + self.stock_dim + 1] -= sell_num_shares
                    self.cost += (
                        self.state[index + 1]
                        * sell_num_shares
                        * self.sell_cost_pct[index]
                    )
                    self.trades += 1
                    #if sell_num_shares >0:
                    #  print("stocks Sold sell_num_shares", sell_num_shares)
                else:
                    sell_num_shares = 0
            else:
                sell_num_shares = 0
            return sell_num_shares

        # perform sell action based on the sign of the action
        if self.turbulence_threshold is not None:
            if self.turbulence >= self.turbulence_threshold:
                if self.state[index + 1] > 0:
                    # Sell only if the price is > 0 (no missing data in this particular date)
                    # if turbulence goes over threshold, just clear out all positions
                    if self.state[index + self.stock_dim + 1] > 0:
                        # Sell only if current asset is > 0
                        sell_num_shares = self.state[index + self.stock_dim + 1]
                        sell_amount = (
                            self.state[index + 1]
                            * sell_num_shares
                            * (1 - self.sell_cost_pct[index])
                        )
                        # update balance
                        self.state[0] += sell_amount
                        self.state[index + self.stock_dim + 1] = 0
                        self.cost += (
                            self.state[index + 1]
                            * sell_num_shares
                            * self.sell_cost_pct[index]
                        )
                        self.trades += 1
                    else:
                        sell_num_shares = 0
                else:
                    sell_num_shares = 0
            else:
                sell_num_shares = _do_sell_normal()
        else:
            sell_num_shares = _do_sell_normal()
        #print("stocks Sold sell_num_shares", sell_num_shares)
        return sell_num_shares

    def _buy_stock(self, index, action):
        def _do_buy():
            if (self.state[index + 2 * self.stock_dim + 1] != True):  # check if the stock is able to buy
                # if self.state[index + 1] >0:
                # Buy only if the price is > 0 (no missing data in this particular date)
                available_amount = self.state[0] // (self.state[index + 1] * (1 + self.buy_cost_pct[index]))
                # when buying stocks, we should consider the cost of trading when calculating available_amount, or we may be have cash<0
                # print('available_amount:{}'.format(available_amount))
                # update balance
                buy_num_shares = min(available_amount, action)
                buy_amount = (self.state[index + 1] * buy_num_shares * (1 + self.buy_cost_pct[index]))
                self.state[0] -= buy_amount
                self.state[index + self.stock_dim + 1] += buy_num_shares
                self.cost += (self.state[index + 1] * buy_num_shares * self.buy_cost_pct[index])
                self.trades += 1
            else:
                buy_num_shares = 0

            return buy_num_shares

        # perform buy action based on the sign of the action
        if self.turbulence_threshold is None:
            buy_num_shares = _do_buy()
        else:
            if self.turbulence < self.turbulence_threshold:
                buy_num_shares = _do_buy()
            else:
                buy_num_shares = 0
                pass

        return buy_num_shares

    def _make_plot(self):
        plt.plot(self.asset_memory, "r")
        plt.savefig(f"results/account_value_trade_{self.episode}.png")
        plt.close()

    def step(self, actions):
        self.terminal = self.day >= len(self.df.index.unique()) - 1
        if self.terminal:
            # print(f"Episode: {self.episode}")
            if self.make_plots:
                self._make_plot()
            end_total_asset = self.state[0] + sum(np.array(self.state[1 : (self.stock_dim + 1)]) * np.array(self.state[(self.stock_dim + 1) : (self.stock_dim * 2 + 1)]))
            df_total_value = pd.DataFrame(self.asset_memory)
            tot_reward = (self.state[0] + 
                          sum(np.array(self.state[1 : (self.stock_dim + 1)]) 
                          * np.array(self.state[(self.stock_dim + 1) : (self.stock_dim * 2 + 1)])) 
                          - self.asset_memory[0])  # initial_amount is only cash part of our initial asset
            df_total_value.columns = ["account_value"]
            df_total_value["date"] = self.date_memory
            df_total_value["daily_return"] = df_total_value["account_value"].pct_change(1)
            if df_total_value["daily_return"].std() != 0:
                sharpe = ((252**0.5) 
                * df_total_value["daily_return"].mean() 
                / df_total_value["daily_return"].std())
            df_rewards = pd.DataFrame(self.rewards_memory)
            df_rewards.columns = ["account_rewards"]
            df_rewards["date"] = self.date_memory[:-1]
            
            if self.episode % self.print_verbosity == 0:
                print(f"day: {self.day}, episode: {self.episode}")
                print(f"begin_total_asset: {self.asset_memory[0]:0.2f}")
                print(f"end_total_asset: {end_total_asset:0.2f}")
                print(f"total_reward: {tot_reward:0.2f}")
                print(f"total_cost: {self.cost:0.2f}")
                print(f"total_trades: {self.trades}")
                if df_total_value["daily_return"].std() != 0:
                    print(f"Sharpe: {sharpe:0.3f}")
                print("=================================")

            if (self.model_name != "") and (self.mode != ""):
                df_actions = self.save_action_memory()
                df_actions.to_csv("results/actions_{}_{}_{}.csv".format(self.mode, self.model_name, self.iteration))
                df_total_value.to_csv("results/account_value_{}_{}_{}.csv".format(self.mode, self.model_name, self.iteration),index=False,)
                df_rewards.to_csv("results/account_rewards_{}_{}_{}.csv".format(self.mode, self.model_name, self.iteration),index=False,)
                plt.plot(self.asset_memory, "r")
                plt.savefig("results/account_value_{}_{}_{}.png".format(self.mode, self.model_name, self.iteration))
                plt.close()
            return self.state, self.reward, self.terminal, {}

        else:
            #print(actions)
            actions = actions * self.hmax  # actions initially is scaled between 0 to 1
            #print(actions)
            actions = actions.astype(int)  # convert into integer because we can't by fraction of shares

            sell_penalty = 0
            sell_bonus = 0

            if self.turbulence_threshold is not None:
                if self.turbulence >= self.turbulence_threshold:
                    actions = np.array([-self.hmax] * self.stock_dim)
            
            current_price = np.array(self.state[1 : (self.stock_dim + 1)])
            # actions = np.where(((current_price > ( self.avg_price * 0.4  + self.avg_price))& (self.avg_price >0.0)), self.total_stockss*-1,actions  )
            # actions = np.where(((current_price < (self.avg_price - self.avg_price * 0.2))& (self.avg_price >0)), self.total_stockss*-1,actions  )
            
            if self.count > 4:
              # for i in range(0,len(actions)):
              #   current_price  = np.array(self.state[1 : (self.stock_dim + 1)])[i]
              #   avg_price = self.avg_price[i]
              #   if (current_price > (avg_price))  and (avg_price > 0.0):
              #     sell_bonus = sell_bonus + 150
              #     print("profit:", round(current_price - avg_price))
              #   elif current_price < (avg_price)  and avg_price > 0.0:
              #     sell_penalty = sell_penalty - 100
              #     print("Loss:" ,round(avg_price - current_price))

              sell_bonus = (len(np.where(current_price > self.avg_price)) * 100 ) + sell_bonus
              sell_penalty = (len(np.where(current_price < self.avg_price)) * -300 ) + sell_penalty

              actions = np.where(self.total_stockss >0.0, self.total_stockss*-1,0.0)
              actions = np.where(actions >0.0, 0.0,actions)
              self.count = 0
            else:
              # for i in range(0,len(actions)):
              #   current_price  = np.array(self.state[1 : (self.stock_dim + 1)])[i]
              #   avg_price = self.avg_price[i]
              #   #print(avg_price)
              #   if (current_price > ( avg_price * 0.4 + avg_price))  and (avg_price > 0.0):
              #     actions[i] = self.total_stockss[i] * -1
              #     #print("Updated actions")
              #       #self.total_price[i] = 0.0
              #     print(round(current_price) , "|", " avg_price", round(avg_price), "Profit", round(current_price -  avg_price))
              #     sell_bonus = sell_bonus + 50
              #   elif current_price < (avg_price - avg_price * 0.2)  and avg_price > 0.0:
              #     actions[i] = self.total_stockss[i] * -1
              #     print(round(current_price) , "|", " avg_price", round(avg_price),  "loss", round(current_price -  avg_price))
              #     sell_penalty = sell_penalty - 50
              #     #self.total_price[i] = 0.0
                sell_bonus = (len(np.where(((current_price > ( self.avg_price * 0.3  + self.avg_price))& (self.avg_price >0.0)))) * 100 ) + sell_bonus
                sell_penalty = (len(((current_price < (self.avg_price - self.avg_price * 0.1))& (self.avg_price >0))) * - 150 ) + sell_penalty
                
                actions = np.where(((current_price > ( self.avg_price * 0.4  + self.avg_price))& (self.avg_price >0.0)), self.total_stockss*-1,actions  )
                actions = np.where(((current_price < (self.avg_price - self.avg_price * 0.2))& (self.avg_price >0)), self.total_stockss*-1,actions  )





            begin_total_asset = self.state[0] + sum(np.array(self.state[1 : (self.stock_dim + 1)])* np.array(self.state[(self.stock_dim + 1) : (self.stock_dim * 2 + 1)]))
            # print("begin_total_asset:{}".format(begin_total_asset))


            argsort_actions = np.argsort(actions)
            sell_index = argsort_actions[: np.where(actions < 0)[0].shape[0]]
            buy_index = argsort_actions[::-1][: np.where(actions > 0)[0].shape[0]]

            for index in sell_index:
              actions[index] = self._sell_stock(index, actions[index]) * (-1)

            for index in buy_index:
                actions[index] = self._buy_stock(index, actions[index])


            self.actions_memory.append(actions)

            # state: s -> s+1
            self.day += 1
            self.count += 1
            self.data = self.df.loc[self.day, :]
            if self.turbulence_threshold is not None:
                if len(self.df.tic.unique()) == 1:
                    self.turbulence = self.data[self.risk_indicator_col]
                elif len(self.df.tic.unique()) > 1:
                    self.turbulence = self.data[self.risk_indicator_col].values[0]



            
            recent_price = copy.deepcopy(self.state[1 : (self.stock_dim + 1)])
            recent_buy_sell = copy.deepcopy(actions)
            recent_buy_sell_price = np.where(recent_buy_sell != 0.0, recent_buy_sell * recent_price, 0.0)


            
            self.total_stockss = copy.deepcopy(recent_buy_sell) + copy.deepcopy(self.total_stockss)
            self.total_price = copy.deepcopy(recent_buy_sell_price)+ copy.deepcopy(self.total_price)
            
            def _def_find_avg(x):
                index, val = x
                recent_action, recent_price = val
                if recent_action > 0:
                  self.portfolios[index].buy(recent_action, recent_price)
                elif recent_action < 0:
                  self.portfolios[index].sell(abs(recent_action))
                return round(self.portfolios[index].get_avg_cost(), 2)
              
              
            indexed = enumerate(zip(recent_buy_sell, recent_price))
            
            self.avg_price = np.fromiter(map(_def_find_avg, indexed, ), dtype=float)
            
            #print(self.avg_price )
            # self.avg_price = np.divide(self.total_price,
            #                            self.total_stockss,
            #                            out=np.zeros_like(self.total_price),
            #                            #where=((recent_buy_sell!=0.0)) &(self.total_stockss >0.0) )
            #                            where=recent_buy_sell!=0.0)

            self.avg_price = np.where(~np.isfinite(self.avg_price), 0.0, self.avg_price)
            self.avg_price = np.where(self.avg_price<0.0, 0.0, self.avg_price)

            self.avg_price = np.where(self.total_stockss <=0.0,0.0,self.avg_price)


            # new_df = pd.DataFrame([recent_price])
            # self.frame = pd.concat([self.frame, pd.DataFrame(new_df)],ignore_index=True,axis = 0)
            
            # new_df = pd.DataFrame([recent_buy_sell_price])
            # self.frame = pd.concat([self.frame, pd.DataFrame(new_df)],ignore_index=True,axis = 0)
            
            # new_df = pd.DataFrame([self.total_stockss])
            # self.frame = pd.concat([self.frame, pd.DataFrame(new_df)],ignore_index=True,axis = 0)

            # new_df = pd.DataFrame([actions])
            # self.frame = pd.concat([self.frame, pd.DataFrame(new_df)],ignore_index=True,axis = 0)

            # new_df = pd.DataFrame([self.total_price])
            # self.frame = pd.concat([self.frame, pd.DataFrame(new_df)],ignore_index=True,axis = 0)

            # new_df = pd.DataFrame([self.avg_price])
            # self.frame = pd.concat([self.frame, pd.DataFrame(new_df)],ignore_index=True,axis = 0)

            # self.frame.to_csv("test.csv")
            
            
            
            # dx = np.where(((self.avg_price < 0.00)))
            # print(dx)
            #if len(dx) > 0:
            #print(self.total_stockss)
            #print(self.total_price )
            #print(self.avg_price)


            




            # self.total_price = np.add(recent_buy_sell_price,
            #                           self.total_price,
            #                           out=np.zeros_like(self.total_price),
            #                                where=(recent_buy_sell!=0.0  )
            
            # #self.total_price = np.where(self.total_price<0.0, 0.0, self.total_price)
            # self.total_stockss = actions + self.total_stockss
            
            # self.avg_price = np.divide(self.total_price,
            #                            self.total_stockss,
            #                            out=np.zeros_like(self.total_price),
            #                            where=recent_buy_sell!=0.0)
          
            # print("Total Stocks")
            # print(self.total_stockss)
            # print("Price")
            # print(recent_price)

            # print("Avg Price")
            # print(self.avg_price)
            # print("Total Price")
            # print(self.total_price)


            self.state = self._update_state()

            end_total_asset = self.state[0] + sum(
                np.array(self.state[1 : (self.stock_dim + 1)])
                * np.array(self.state[(self.stock_dim + 1) : (self.stock_dim * 2 + 1)])
            )
            self.asset_memory.append(end_total_asset)
            self.date_memory.append(self._get_date())
            self.reward = sell_bonus - sell_penalty
            self.rewards_memory.append(self.reward)
            self.reward = self.reward * self.reward_scaling
            self.state_memory.append(
                self.state
            )  # add current state in state_recorder for each step

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        # initiate state
        self.state = self._initiate_state()

        if self.initial:
            self.asset_memory = [
                self.initial_amount
                + np.sum(
                    np.array(self.num_stock_shares)
                    * np.array(self.state[1 : 1 + self.stock_dim])
                )
            ]
        else:
            previous_total_asset = self.previous_state[0] + sum(
                np.array(self.state[1 : (self.stock_dim + 1)])
                * np.array(
                    self.previous_state[(self.stock_dim + 1) : (self.stock_dim * 2 + 1)]
                )
            )
            self.asset_memory = [previous_total_asset]

        self.day = 0
        self.data = self.df.loc[self.day, :]
        self.turbulence = 0
        self.cost = 0
        self.trades = 0
        self.terminal = False
        # self.iteration=self.iteration
        self.rewards_memory = []
        self.actions_memory = []
        self.date_memory = [self._get_date()]
        if self.total_price[5] > 0.0:
          dd = pd.DataFrame(data = self.avg_price)
          dd.to_csv("avg")
          dd = pd.DataFrame(data = self.total_price)
          dd.to_csv("tp")
        
        self.episode += 1
        self.total_price = np.array([0.0] * 29)
        self.avg_price = np.array([0.0] * 29)
        self.total_stockss = np.array([0.0] * 29)
        self.portfolios = [Portfolio() for _ in range(29)]
        self.count = 0
        #print("Resteting Account")
        return self.state

    def render(self, mode="human", close=False):
        return self.state

    def _initiate_state(self):
        if self.initial:
            # For Initial State
            if len(self.df.tic.unique()) > 1:
                # for multiple stock
                state = (
                    [self.initial_amount]
                    + self.data.close.values.tolist()
                    + self.num_stock_shares
                    + sum(
                        (
                            self.data[tech].values.tolist()
                            for tech in self.tech_indicator_list
                        ),
                        [],
                    )
                )  # append initial stocks_share to initial state, instead of all zero
            else:
                # for single stock
                state = (
                    [self.initial_amount]
                    + [self.data.close]
                    + [0] * self.stock_dim
                    + sum(([self.data[tech]] for tech in self.tech_indicator_list), [])
                )
        else:
            # Using Previous State
            if len(self.df.tic.unique()) > 1:
                # for multiple stock
                state = (
                    [self.previous_state[0]]
                    + self.data.close.values.tolist()
                    + self.previous_state[
                        (self.stock_dim + 1) : (self.stock_dim * 2 + 1)
                    ]
                    + sum(
                        (
                            self.data[tech].values.tolist()
                            for tech in self.tech_indicator_list
                        ),
                        [],
                    )
                )
            else:
                # for single stock
                state = (
                    [self.previous_state[0]]
                    + [self.data.close]
                    + self.previous_state[
                        (self.stock_dim + 1) : (self.stock_dim * 2 + 1)
                    ]
                    + sum(([self.data[tech]] for tech in self.tech_indicator_list), [])
                )
        return state

    def _update_state(self):
        if len(self.df.tic.unique()) > 1:
            # for multiple stock
            state = (
                [self.state[0]]
                + self.data.close.values.tolist()
                + list(self.state[(self.stock_dim + 1) : (self.stock_dim * 2 + 1)])
                + sum((self.data[tech].values.tolist() for tech in self.tech_indicator_list),[],))

        else:
            # for single stock
            state = ([self.state[0]] + [self.data.close] 
                     + list(self.state[(self.stock_dim + 1) : (self.stock_dim * 2 + 1)])
                     + sum(([self.data[tech]] for tech in self.tech_indicator_list), []))

        return state

    def _get_date(self):
        if len(self.df.tic.unique()) > 1:
            date = self.data.date.unique()[0]
        else:
            date = self.data.date
        return date


    def save_asset_memory(self):
        date_list = self.date_memory
        asset_list = self.asset_memory
        # print(len(date_list))
        # print(len(asset_list))
        df_account_value = pd.DataFrame({"date": date_list, "account_value": asset_list})
        return df_account_value

    def save_action_memory(self):
        if len(self.df.tic.unique()) > 1:
            # date and close price length must match actions length
            date_list = self.date_memory[:-1]
            df_date = pd.DataFrame(date_list)
            df_date.columns = ["date"]

            action_list = self.actions_memory
            df_actions = pd.DataFrame(action_list)
            df_actions.columns = self.data.tic.values
            df_actions.index = df_date.date
            # df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        else:
            date_list = self.date_memory[:-1]
            action_list = self.actions_memory
            df_actions = pd.DataFrame({"date": date_list, "actions": action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [167]:
# class CustomStockTradingEnv(StockTradingEnv):
#     def step(self, action):
#         obs, reward, done, info = super().step(action)
#         np.where(self.total_stockss >0.0, self.total_stockss*-1,0.0)
#         begin_total_asset = self.state[0] + sum(np.array(self.state[1 : (self.stock_dim + 1)])* np.array(self.state[(self.stock_dim + 1) : (self.stock_dim * 2 + 1)]))
            
#         return obs, reward, done, info

In [168]:
stock_dimension = len(init_train_data.tic.unique())
state_space = 1 + 2 * stock_dimension + len(config.INDICATORS) * stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

Stock Dimension: 29, State Space: 291


In [169]:
#Defining the environment kwargs

initial_amount = 50000
env_kwargs = {
    "hmax": 100,
    "initial_amount": initial_amount,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": config.INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

In [170]:
#Instantiate the training gym compatible environment
e_train_gym = GokuEnv(df = init_train_data, **env_kwargs)
env_train, _ = e_train_gym.get_sb_env()
agent = DRLAgent(env = env_train)

/usr/local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [171]:
#Instantiate the trading environment
e_trade_gym = GokuEnv(df = init_trade_data, turbulence_threshold = None, **env_kwargs)

# Trade performance code
The following code calculates trade performance metrics, which are then used as an objective for optimizing hyperparameter values.

There are several available metrics. In this tutorial, the default choice is the ratio of average value of winning to losing trades.

In [172]:

#Main method
# Calculates Trade Performance for Objective
# Called from objective method
# Returns selected trade perf metric(s)
# Requires actions and associated prices

def calc_trade_perf_metric(df_actions, 
                           df_prices_trade,
                           tp_metric,
                           dbg=False):
  
    df_actions_p, df_prices_p, tics = prep_data(df_actions.copy(),
                                                df_prices_trade.copy())
    # actions predicted by trained model on trade data
    df_actions_p.to_csv('df_actions.csv') 
    df_prices_p.to_csv('df_price.csv') 

    
    # Confirms that actions, prices and tics are consistent
    df_actions_s, df_prices_s, tics_prtfl = \
        sync_tickers(df_actions_p.copy(),df_prices_p.copy(),tics)
    
    # copy to ensure that tics from portfolio remains unchanged
    tics = tics_prtfl.copy()
    
    # Analysis is performed on each portfolio ticker
    perf_data= collect_performance_data(df_actions_s, df_prices_s, tics)
    # profit/loss for each ticker
    pnl_all = calc_pnl_all(perf_data, tics)
    # values for trade performance metrics
    perf_results = calc_trade_perf(pnl_all)
    df = pd.DataFrame.from_dict(perf_results, orient='index')
    
    # calculate and return trade metric value as objective
    m = calc_trade_metric(df,tp_metric)
    print(f'Ratio Avg Win/Avg Loss: {m}')
    k = str(len(tpm_hist)+1)
    # save metric value
    tpm_hist[k] = m
    return m


# Supporting methods
def calc_trade_metric(df,metric='avgwl'):
    '''# trades', '# wins', '# losses', 'wins total value', 'wins avg value',
       'losses total value', 'losses avg value'''
    # For this tutorial, the only metric available is the ratio of 
    #  average values of winning to losing trades. Others are in development.
    
    # some test cases produce no losing trades.
    # The code below assigns a value as a multiple of the highest value during
    # previous hp optimization runs. If the first run experiences no losses,
    # a fixed value is assigned for the ratio
    tpm_mult = 1.0
    avgwl_no_losses = 25
    if metric == 'avgwl':
        if sum(df['# losses']) == 0:
          try:
            return max(tpm_hist.values())*tpm_mult
          except ValueError:
            return avgwl_no_losses
        df.to_csv("winloss.csv")
        avg_w = sum(df['wins total value'])/sum(df['# wins'])
        #print(avg_w)
        avg_l = sum(df['losses total value'])/sum(df['# losses'])
        #print(avg_l)
        m = abs(avg_w/avg_l)

    return m


def prep_data(df_actions,
              df_prices_trade):
    
    df=df_prices_trade[['date','close','tic']]
    df['Date'] = pd.to_datetime(df['date'])
    df = df.set_index('Date')
    # set indices on both df to datetime
    idx = pd.to_datetime(df_actions.index, infer_datetime_format=True)
    df_actions.index=idx
    tics = np.unique(df.tic)
    n_tics = len(tics)
    print(f'Number of tickers: {n_tics}')
    print(f'Tickers: {tics}')
    dategr = df.groupby('tic')
    p_d={t:dategr.get_group(t).loc[:,'close'] for t in tics}
    df_prices = pd.DataFrame.from_dict(p_d)
    df_prices.index = df_prices.index.normalize()
    return df_actions, df_prices, tics


# prepares for integrating action and price files
def link_prices_actions(df_a,
                        df_p):
    cols_a = [t + '_a' for t in df_a.columns]
    df_a.columns = cols_a
    cols_p = [t + '_p' for t in df_p.columns]
    df_p.columns = cols_p
    return df_a, df_p


def sync_tickers(df_actions,df_tickers_p,tickers):
    # Some DOW30 components may not be included in portfolio
    # passed tickers includes all DOW30 components
    # actions and ticker files may have different length indices
    if len(df_actions) != len(df_tickers_p):
      msng_dates = set(df_actions.index)^set(df_tickers_p.index)
      try:
        #assumption is prices has one additional timestamp (row)
        df_tickers_p.drop(msng_dates,inplace=True)
      except:
        df_actions.drop(msng_dates,inplace=True)
    df_actions, df_tickers_p = link_prices_actions(df_actions,df_tickers_p)
    # identify any DOW components not in portfolio
    t_not_in_a = [t for t in tickers if t + '_a' not in list(df_actions.columns)]
  
    # remove t_not_in_a from df_tickers_p
    drop_cols = [t + '_p' for t in t_not_in_a]
    df_tickers_p.drop(columns=drop_cols,inplace=True)
    
    # Tickers in portfolio
    tickers_prtfl = [c.split('_')[0] for c in df_actions.columns]
    return df_actions,df_tickers_p, tickers_prtfl

def collect_performance_data(dfa,dfp,tics, dbg=False):
    
    perf_data = {}
    # In current version, files columns include secondary identifier
    for t in tics:
        # actions: purchase/sale of DOW equities
        acts = dfa['_'.join([t,'a'])].values
        # ticker prices
        prices = dfp['_'.join([t,'p'])].values
        # market value of purchases/sales
        tvals_init = np.multiply(acts,prices)
        d={'actions':acts, 'prices':prices,'init_values':tvals_init}
        perf_data[t]=d

    return perf_data


def calc_pnl_all(perf_dict, tics_all):
    # calculate profit/loss for each ticker
    print(f'Calculating profit/loss for each ticker')
    pnl_all = {}
    for tic in tics_all:
        pnl_t = []
        tic_data = perf_dict[tic]
        init_values = tic_data['init_values']
        acts = tic_data['actions']
        prices = tic_data['prices']
        cs = np.cumsum(acts)
        args_s = [i + 1 for i in range(len(cs) - 1) if cs[i + 1] < cs[i]]
        # tic actions with no sales
        if not args_s:
            pnl = complete_calc_buyonly(acts, prices, init_values)
            pnl_all[tic] = pnl
            continue
        # copy acts: acts_rev will be revised based on closing/reducing init positions
        pnl_all = execute_position_sales(tic,acts,prices,args_s,pnl_all)

    return pnl_all


def complete_calc_buyonly(actions, prices, init_values):
    # calculate final pnl for each ticker assuming no sales
    fnl_price = prices[-1]
    final_values = np.multiply(fnl_price, actions)
    pnl = np.subtract(final_values, init_values)
    return pnl


def execute_position_sales(tic,acts,prices,args_s,pnl_all):
  # calculate final pnl for each ticker with sales
    pnl_t = []
    acts_rev = acts.copy()
    # location of sales transactions
    for s in args_s:  # s is scaler
        # price_s = [prices[s]]
        act_s = [acts_rev[s]]
        args_b = [i for i in range(s) if acts_rev[i] > 0]
        prcs_init_trades = prices[args_b]
        acts_init_trades = acts_rev[args_b]
  
        # update actions for sales
        # reduce/eliminate init values through trades
        # always start with earliest purchase that has not been closed through sale
        # selectors for purchase and sales trades
        # find earliest remaining purchase
        arg_sel = min(args_b)
        # sel_s = len(acts_trades) - 1

        # closing part/all of earliest init trade not yet closed
        # sales actions are negative
        # in this test case, abs_val of init and sales share counts are same
        # zero-out sales actions
        # market value of sale
        # max number of shares to be closed: may be less than # originally purchased
        acts_shares = min(abs(act_s.pop()), acts_rev[arg_sel])

        # mv of shares when purchased
        mv_p = abs(acts_shares * prices[arg_sel])
        # mv of sold shares
        mv_s = abs(acts_shares * prices[s])

        # calc pnl
        pnl = mv_s - mv_p
        # reduce init share count
        # close all/part of init purchase
        acts_rev[arg_sel] -= acts_shares
        acts_rev[s] += acts_shares
        # calculate pnl for trade
        # value of associated purchase
        
        # find earliest non-zero positive act in acts_revs
        pnl_t.append(pnl)
    
    pnl_op = calc_pnl_for_open_positions(acts_rev, prices)
    #pnl_op is list
    # add pnl_op results (if any) to pnl_t (both lists)
    pnl_t.extend(pnl_op)
    #print(f'Total pnl for {tic}: {np.sum(pnl_t)}')
    pnl_all[tic] = np.array(pnl_t)
    return pnl_all


def calc_pnl_for_open_positions(acts,prices):
    # identify any positive share values after accounting for sales
    pnl = []
    fp = prices[-1] # last price
    open_pos_arg = np.argwhere(acts>0)
    if len(open_pos_arg)==0:return pnl # no open positions

    mkt_vals_open = np.multiply(acts[open_pos_arg], prices[open_pos_arg])
    # mkt val at end of testing period
    # treat as trades for purposes of calculating pnl at end of testing period
    mkt_vals_final = np.multiply(fp, acts[open_pos_arg])
    pnl_a = np.subtract(mkt_vals_final, mkt_vals_open)
    #convert to list
    pnl = [i[0] for i in pnl_a.tolist()]
    #print(f'Market value of open positions at end of testing {pnl}')
    return pnl


def calc_trade_perf(pnl_d):
    # calculate trade performance metrics
    perf_results = {}
    for t,pnl in pnl_d.items():
        wins = pnl[pnl>0]  # total val
        losses = pnl[pnl<0]
        n_wins = len(wins)
        n_losses = len(losses)
        n_trades = n_wins + n_losses
        wins_val = np.sum(wins)
        losses_val = np.sum(losses)
        wins_avg = 0 if n_wins==0 else np.mean(wins)
        #print(f'{t} n_wins: {n_wins} n_losses: {n_losses}')
        losses_avg = 0 if n_losses==0 else np.mean(losses)
        d = {'# trades':n_trades,'# wins':n_wins,'# losses':n_losses,
             'wins total value':wins_val, 'wins avg value':wins_avg,
             'losses total value':losses_val, 'losses avg value':losses_avg,}
        perf_results[t] = d
    return perf_results

# Tuning hyperparameters using Optuna

In [173]:
def sample_ddpg_params(trial:optuna.Trial):
  # Size of the replay buffer
  buffer_size = trial.suggest_categorical("buffer_size", [int(1e4), int(1e5), int(1e6)])
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1)
  batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256, 512])
  
  return {"buffer_size": buffer_size,
          "learning_rate":learning_rate,
          "batch_size":batch_size}

In [189]:
# Set Variables
## Fixed
tpm_hist = {}  # record tp metric values for trials
tp_metric = 'avgwl'  # specified trade_param_metric: ratio avg value win/loss
## Settable by User
n_trials = 100  # number of HP optimization runs
total_timesteps = 2000 # per HP optimization run
## Logging callback params
lc_threshold=3e-5
lc_patience=99
lc_trial_number=10

OPTIONAL CODE FOR SAMPLING HYPERPARAMETERS

Replace current call in function objective with

hyperparameters = sample_ddpg_params_all(trial)

In [198]:
def sample_ddpg_params_all(trial:optuna.Trial,
                           # fixed values from previous study
                           learning_rate=0.0103,
                           batch_size=128,
                           buffer_size=int(1e6)):

    buffer_size = trial.suggest_int("buffer_size", int(1e4), int(1e6))
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1)
    batch_size = trial.suggest_int("batch_size", 32, 1024)
    gamma = trial.suggest_float("gamma", 0.1, 2)
    
    # Polyak coeff
    tau = trial.suggest_float("tau", 1e-5, 1)
    
    train_freq = trial.suggest_int("train_freq", 64, 2048)
    gradient_steps = trial.suggest_int("gradient_steps", 64, 2048)
    
    #noise_type = trial.suggest_categorical("noise_type", ["ornstein-uhlenbeck", "normal", None])
    #noise_std = trial.suggest_categorical("noise_std", [.0,.1,.2,.3, .4, .5, .6,.8] )

    # NOTE: Add "verybig" to net_arch when tuning HER (see TD3)
    net_arch = trial.suggest_categorical("net_arch", ["verysmall","small","medium", "big", "verybig",
                                                      "same_verysmall","same_small","same_medium", "same_big", "same_verybig"])
    # activation_fn = trial.suggest_categorical('activation_fn', [nn.Tanh, nn.ReLU, nn.ELU, nn.LeakyReLU])

    net_arch = {
        "verysmall": [64, 32],
        "small": [128, 64],
        "medium": [256, 512],
        "big": [512, 1024],
        "verybig": [2028, 1024],
        "same_verysmall": [64, 64],
        "same_small": [128, 128],
        "same_medium": [256, 256],
        "same_big": [512, 512],
        "same_verybig": [1024, 1024],
    }[net_arch]
  
    hyperparams = {
        "batch_size": batch_size,
        "buffer_size": buffer_size,
        "gamma": gamma,
        "gradient_steps": gradient_steps,
        "learning_rate": learning_rate,
        "tau": tau,
        "train_freq": train_freq,
        #"noise_std": noise_std,
        #"noise_type": noise_type,
        
        "policy_kwargs": dict(net_arch=net_arch)
    }
    return hyperparams

# Callbacks


1. The callback will terminate if the improvement margin is below certain point
2. It will terminate after certain number of trial_number are reached, not before that
3. It will hold its patience to reach the threshold

In [199]:
class LoggingCallback:
    def __init__(self,threshold,trial_number,patience):
      '''
      threshold:int tolerance for increase in objective
      trial_number: int Prune after minimum number of trials
      patience: int patience for the threshold
      '''
      self.threshold = threshold
      self.trial_number  = trial_number
      self.patience = patience
      print(f'Callback threshold {self.threshold}, \
            trial_number {self.trial_number}, \
            patience {self.patience}')
      self.cb_list = [] #Trials list for which threshold is reached
    def __call__(self,study:optuna.study, frozen_trial:optuna.Trial):
      #Setting the best value in the current trial
      study.set_user_attr("previous_best_value", study.best_value)
      
      #Checking if the minimum number of trials have pass
      if frozen_trial.number >self.trial_number:
          previous_best_value = study.user_attrs.get("previous_best_value",None)
          #Checking if the previous and current objective values have the same sign
          if previous_best_value * study.best_value >=0:
              #Checking for the threshold condition
              if abs(previous_best_value-study.best_value) < self.threshold: 
                  self.cb_list.append(frozen_trial.number)
                  #If threshold is achieved for the patience amount of time
                  if len(self.cb_list)>self.patience:
                      print('The study stops now...')
                      print('With number',frozen_trial.number ,'and value ',frozen_trial.value)
                      print('The previous and current best values are {} and {} respectively'
                              .format(previous_best_value, study.best_value))
                      study.stop()

In [200]:
#Calculate the Sharpe ratio
#This is our objective for tuning
def calculate_sharpe(df):
  df['daily_return'] = df['account_value'].pct_change(1)
  if df['daily_return'].std() !=0:
    sharpe = df['daily_return'].sum()
    #sharpe = (252**0.5)*df['daily_return'].mean()/ \
      #    df['daily_return'].std()
    return sharpe
  else:
    return 0

In [201]:
from IPython.display import clear_output
import sys   

os.makedirs("models",exist_ok=True)

def objective(trial:optuna.Trial):
  #Trial will suggest a set of hyperparamters from the specified range

  # Optional to optimize larger set of parameters
  # hyperparameters = sample_ddpg_params_all(trial)
  
  # Optimize buffer size, batch size, learning rate
  hyperparameters = sample_ddpg_params_all(trial)
  print(f'Hyperparameters from objective: {hyperparameters.keys()}')
  policy_kwargs = None  # default
  if 'policy_kwargs' in hyperparameters.keys():
    policy_kwargs = hyperparameters['policy_kwargs']
    del hyperparameters['policy_kwargs']
    print(f'Policy keyword arguments {policy_kwargs}')
  model_ddpg = agent.get_model("ddpg",
                               policy_kwargs = policy_kwargs,
                               model_kwargs = hyperparameters )
  
  #You can increase it for better comparison
  trained_ddpg = agent.train_model(model=model_ddpg,
                                   tb_log_name="ddpg",
                                   total_timesteps=total_timesteps)
  trained_ddpg.save('models/ddpg_{}.pth'.format(trial.number))
  clear_output(wait=True)
  
  #For the given hyperparamters, determine the account value in the trading period
  df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_ddpg, 
    environment = e_trade_gym)
 
  # Calculate trade performance metric
  # Currently ratio of average win and loss market values
  #tpm = calc_trade_perf_metric(df_actions,init_trade_data,tp_metric)
  tpm = calculate_sharpe(df_account_value)
  return tpm

#Create a study object and specify the direction as 'maximize'
#As you want to maximize sharpe
#Pruner stops not promising iterations
#Use a pruner, else you will get error related to divergence of model
#You can also use Multivariate samplere
#sampler = optuna.samplers.TPESampler(multivarite=True,seed=42)
sampler = optuna.samplers.TPESampler()

study = optuna.create_study(study_name="ddpg_study",direction='maximize',
                            sampler = sampler, pruner=optuna.pruners.HyperbandPruner())

logging_callback = LoggingCallback(threshold=lc_threshold,
                                   patience=lc_patience,
                                   trial_number=lc_trial_number)
#You can increase the n_trials for a better search space scanning
study.optimize(objective, n_trials=n_trials,catch=(ValueError,),callbacks=[logging_callback])

/usr/local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
[I 2023-05-05 18:52:33,248] Trial 24 finished with value: -0.06508048125215726 and parameters: {'buffer_size': 575691, 'learning_rate': 0.7543131790393993, 'batch_size': 685, 'gamma': 0.8641840343624055, 'tau': 0.7804816665753948, 'train_freq': 1470, 'gradient_steps': 570, 'net_arch': 'verysmall'}. Best is trial 8 with value: 0.022023361310304024.


hit end!
Hyperparameters from objective: dict_keys(['batch_size', 'buffer_size', 'gamma', 'gradient_steps', 'learning_rate', 'tau', 'train_freq', 'policy_kwargs'])
Policy keyword arguments {'net_arch': [64, 64]}
{'batch_size': 538, 'buffer_size': 452751, 'gamma': 1.05693974574143, 'gradient_steps': 1254, 'learning_rate': 0.659810794353715, 'tau': 0.6243565269805503, 'train_freq': 1043}
Using cuda device


[W 2023-05-05 18:53:13,665] Trial 25 failed with parameters: {'buffer_size': 452751, 'learning_rate': 0.659810794353715, 'batch_size': 538, 'gamma': 1.05693974574143, 'tau': 0.6243565269805503, 'train_freq': 1043, 'gradient_steps': 1254, 'net_arch': 'same_verysmall'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-201-26bf33718e81>", line 25, in objective
    trained_ddpg = agent.train_model(model=model_ddpg,
  File "/usr/local/lib/python3.10/site-packages/finrl/agents/stablebaselines3/models.py", line 103, in train_model
    model = model.learn(
  File "/usr/local/lib/python3.10/site-packages/stable_baselines3/ddpg/ddpg.py", line 123, in learn
    return super().learn(
  File "/usr/local/lib/python3.10/site-packages/stable_baselines3/td3/td3.py", line 222, in learn
    return super().learn(


KeyboardInterrupt: ignored

In [202]:
joblib.dump(study, "final_ddpg_study__.pkl")

['final_ddpg_study__.pkl']

In [203]:
#Get the best hyperparamters
print('Hyperparameters after tuning',study.best_params)
print('Hyperparameters before tuning',config.DDPG_PARAMS)

Hyperparameters after tuning {'buffer_size': 336164, 'learning_rate': 0.8053031766913865, 'batch_size': 835, 'gamma': 1.020257418732807, 'tau': 0.7756058960200708, 'train_freq': 294, 'gradient_steps': 1393, 'net_arch': 'big'}
Hyperparameters before tuning {'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}


In [204]:
study.best_trial

FrozenTrial(number=8, state=TrialState.COMPLETE, values=[0.022023361310304024], datetime_start=datetime.datetime(2023, 5, 5, 18, 31, 37, 929293), datetime_complete=datetime.datetime(2023, 5, 5, 18, 33, 20, 184724), params={'buffer_size': 336164, 'learning_rate': 0.8053031766913865, 'batch_size': 835, 'gamma': 1.020257418732807, 'tau': 0.7756058960200708, 'train_freq': 294, 'gradient_steps': 1393, 'net_arch': 'big'}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'buffer_size': IntDistribution(high=1000000, log=False, low=10000, step=1), 'learning_rate': FloatDistribution(high=1.0, log=False, low=1e-05, step=None), 'batch_size': IntDistribution(high=1024, log=False, low=32, step=1), 'gamma': FloatDistribution(high=2.0, log=False, low=0.1, step=None), 'tau': FloatDistribution(high=1.0, log=False, low=1e-05, step=None), 'train_freq': IntDistribution(high=2048, log=False, low=64, step=1), 'gradient_steps': IntDistribution(high=2048, log=False, low=64, step=1), 'net_

In [205]:
from stable_baselines3 import DDPG
tuned_model_ddpg = DDPG.load('models/ddpg_{}.pth'.format(study.best_trial.number),env=env_train)

In [206]:
#Trading period account value with tuned model
df_account_value_tuned, df_actions_tuned = DRLAgent.DRL_prediction(
    model=tuned_model_ddpg, 
    environment = e_trade_gym)

/usr/local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


hit end!


In [207]:
calc_trade_perf_metric(df_actions_tuned,init_trade_data,tp_metric)

Number of tickers: 29
Tickers: ['AAPL' 'AMGN' 'AXP' 'BA' 'CAT' 'CRM' 'CSCO' 'CVX' 'DIS' 'GS' 'HD' 'HON' 'IBM' 'INTC' 'JNJ' 'JPM' 'KO' 'MCD' 'MMM' 'MRK' 'MSFT' 'NKE' 'PG' 'TRV' 'UNH' 'V' 'VZ' 'WBA' 'WMT']
Calculating profit/loss for each ticker
Ratio Avg Win/Avg Loss: 1.0361022153493156


<ipython-input-172-3b75d6d122ec>:79: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  idx = pd.to_datetime(df_actions.index, infer_datetime_format=True)


1.0361022153493156

In [208]:
df_account_value_tuned

,date,account_value
0,2022-05-02,50000.000000
1,2022-05-03,49772.748732
2,2022-05-04,50910.182684
3,2022-05-05,49472.952712
4,2022-05-06,48962.231761
...,...,...
244,2023-04-21,47419.781027
245,2023-04-24,47132.599676
246,2023-04-25,47085.490347
247,2023-04-26,48862.496285


In [158]:
#Backtesting with our pruned model
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all_tuned = backtest_stats(account_value=df_account_value_tuned)
perf_stats_all_tuned = pd.DataFrame(perf_stats_all_tuned)
perf_stats_all_tuned.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_tuned_"+now+'.csv')

==============Get Backtest Results===========
Annual return          0.036225
Cumulative returns     0.035786
Annual volatility      0.173877
Sharpe ratio           0.292291
Calmar ratio           0.252543
Stability              0.273118
Max drawdown          -0.143441
Omega ratio            1.053515
Sortino ratio          0.414665
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.270559
Daily value at risk   -0.021705
dtype: float64


In [ ]:
#Now train with not tuned hyperaparameters
#Default config.ddpg_PARAMS
non_tuned_model_ddpg = agent.get_model("ddpg",model_kwargs = config.DDPG_PARAMS )
trained_ddpg = agent.train_model(model=non_tuned_model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000)

Streaming output truncated to the last 5000 lines.
328 |  avg_price 141.02 Profit 186.94868896484368
43 |  avg_price 38.64 Profit 3.931975708007812
100 |  avg_price 80.08 Profit 20.101640625
225 |  avg_price 171.31 Profit 53.445889892578066
50 |  avg_price 41.82 Profit 8.351058654785163
77 |  avg_price 68.69 Profit 8.185991821289065
98 |  avg_price 84.79 Profit 13.078003845214837
328 |  avg_price 141.02 Profit 187.24690673828124
43 |  avg_price 38.64 Profit 4.690738067626953
100 |  avg_price 80.08 Profit 20.290918273925783
228 |  avg_price 171.31 Profit 56.3236669921875
51 |  avg_price 41.82 Profit 8.893157653808593
77 |  avg_price 68.69 Profit 8.16881042480469
99 |  avg_price 84.79 Profit 14.30014892578123
330 |  avg_price 141.02 Profit 189.36430786132811
43 |  avg_price 38.64 Profit 4.528136596679687
100 |  avg_price 80.08 Profit 19.981180114746095
228 |  avg_price 171.31 Profit 56.7413916015625
51 |  avg_price 41.82 Profit 8.775706939697272
77 |  avg_price 68.69 Profit 8.52945190429

In [ ]:
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_ddpg, 
    environment = e_trade_gym)

221 |  avg_price 232.33 loss -11.550001220703138
303 |  avg_price 316.46 loss -13.860421142578105
50 |  avg_price 53.12 loss -2.8463793945312474
235 |  avg_price 246.66 loss -11.894161376953122
218 |  avg_price 229.08 loss -11.531507568359359
223 |  avg_price 232.33 loss -9.690000610351575
51 |  avg_price 53.12 loss -2.480713958740232
239 |  avg_price 246.66 loss -7.935558471679684
221 |  avg_price 229.08 loss -8.144483642578138
51 |  avg_price 53.12 loss -1.7025157165527247
239 |  avg_price 246.66 loss -7.886425170898406
224 |  avg_price 232.33 loss -7.9100018310547
300 |  avg_price 310.64 loss -10.474594726562486
222 |  avg_price 229.08 loss -7.3051434326172
224 |  avg_price 210.54 Profit 13.239998779296883
227 |  avg_price 210.54 Profit 16.4500054931641
301 |  avg_price 310.64 loss -10.102371826171861
228 |  avg_price 210.54 Profit 17.170006713867195
230 |  avg_price 210.54 Profit 19.139992675781258
251 |  avg_price 232.33 Profit 18.369996948242175
226 |  avg_price 210.54 Profit 15.

In [ ]:
#Backtesting for not tuned hyperparamters
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
# perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

==============Get Backtest Results===========
Annual return         -0.093379
Cumulative returns    -0.040397
Annual volatility      0.113727
Sharpe ratio          -0.813524
Calmar ratio          -1.527834
Stability              0.006414
Max drawdown          -0.061119
Omega ratio            0.871818
Sortino ratio         -1.035397
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.710234
Daily value at risk   -0.014695
dtype: float64


In [ ]:
#You can see with trial, our sharpe ratio is increasing
#Certainly you can afford more number of trials for further optimization
from optuna.visualization import plot_optimization_history
plot_optimization_history(study)

In [ ]:
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

In [ ]:
#Hyperparamters importance
#Ent_coef is the most important
plot_param_importances(study)